In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # matplotlib is for drawing graphs
import matplotlib.colors as colors
from sklearn.utils import resample # downsample the dataset
from sklearn.model_selection import train_test_split # split data into training and testing sets
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC # this will make a support vector machine for classificaiton
from sklearn.model_selection import GridSearchCV #this will do cross validation
from sklearn.metrics import confusion_matrix # this creates a confusion matrix
from sklearn.decomposition import PCA # to perform PCA to plot the data
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc

In [3]:
df = pd.read_csv("default of credit card clients.txt",sep='\t',header=1)
df.drop('ID',axis=1,inplace=True)
df.rename(columns={'default payment next month': 'DEFAULT'}, inplace=True)
df_no_missing = df.loc[(df['MARRIAGE']!=0) & (df['EDUCATION']!=0)]
df_no_missing.head()
df_no_default = df_no_missing[df_no_missing['DEFAULT']==0]
df_default = df_no_missing[df_no_missing['DEFAULT']==1]
df_no_default_downsampled = resample(df_no_default,replace=False,n_samples=1000,random_state=42)
df_default_downsampled = resample(df_default,replace=False,n_samples=1000,random_state=42)
df_downsampled = pd.concat([df_no_default_downsampled,df_default_downsampled])
X = df_downsampled.drop(['DEFAULT','SEX','EDUCATION','MARRIAGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6'],axis=1).copy()
y = df_downsampled['DEFAULT'].copy()
#X_encoded = pd.get_dummies(X,columns=['SEX','EDUCATION','MARRIAGE','PAY_0','PAY_2','PAY_3','PAY_4','PAY_5','PAY_6']).astype(int)
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)
print("pre-proccessing");

pre-proccessing


In [119]:
print(f"Number of columns: {X_train_scaled.shape[1]}")
input_nodes_num = X_train_scaled.shape[1]
hidden_nodes_num = 8
output_nodes_num = 1
np.random.seed(42)

#hidden-input layer wieghts and biases
input_hidden_weights = np.random.randn(input_nodes_num,hidden_nodes_num)
input_hidden_baises = np.random.randn(hidden_nodes_num)

#hidden-output layer wieghts and biases
hidden_out_weights = np.random.randn(hidden_nodes_num,output_nodes_num)
hidden_out_baises=0

Number of columns: 14


In [133]:
#sum of squared residuals
ssr = 0;
predicated_values = []
hidden_out_baises = 0
for i in range(0,len(X_train_scaled)):
    input_nodes = X_train_scaled[i]
    pred = forward(input_nodes)
    real = y_train.iloc[i]
    predicated_values.append(pred)
    
predicated_values = np.array(predicated_values)
observed_values =  y_train.to_numpy()
ssr = np.sum((predicated_values-observed_values)**2)
print(ssr)
equation1 = r'$\frac{\partial \text{Ssr}}{\partial b} = \frac{\partial \text{Ssr}}{\partial \text{pred}} \cdot \frac{\partial \text{pred}}{\partial b}$'
equation2 = r'$\frac{\partial \text{Ssr}}{\partial \text{pred}} = \sum\frac{\partial (pred - real)^{2}}{\partial \text{pred}} = \sum2(pred - real)$'
equation3 = r'$\frac{\partial \text{pred}}{\partial \text{b}} = \frac{\partial sigmoid(out)}{\partial \text{b}} = \frac{d}{db}sigmoid(out)*1$'    
# Compute dSSR_dpred
db = np.sum(2 * (predicated_values - observed_values)*predicated_values*(1-predicated_values))

print(dSSR_dpred)
#plotEquations(equation1,equation2,equation3)


998758.9288417782
102951.65367016835


In [123]:
def forward(input_nodes):
    #input-nodes
    input_nodes = a;
    #initialize hidden nodes
    hidden_nodes = np.dot(input_nodes,input_hidden_weights)
    hidden_nodes = np.add(hidden_nodes,input_hidden_baises)
    hidden_nodes = relu(hidden_nodes)
    #initialize out node
    output_node = np.dot(hidden_nodes,hidden_out_weights)
    output_node = np.add(output_node,hidden_out_baises)
    predictions = sigmoid(output_node)
    return predictions
    
def plotEquations(q1,q2,q3):
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots()

    ax.axis('off')
    
    ax.text(0.5, 0.7, q1, fontsize=20, ha='center', va='center')
    
    ax.text(0.5, 0.5, q2, fontsize=20, ha='center', va='center')
    
    ax.text(0.5, 0.3, q3, fontsize=20, ha='center', va='center')
    
    plt.savefig('equations.png', bbox_inches='tight', pad_inches=0.1, dpi=300)
    plt.show()
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def relu(x):
    return np.maximum(0, x) 
def sigmoid_derivative(x):
    sigmoid_x = sigmoid(x)
    return sigmoid_x * (1 - sigmoid_x)